In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Loading the data.

In [ ]:
train_data = pd.read_csv('/kaggle/input/project-1-layor-7/train.csv')
train_data.head()

In [ ]:
train_data.shape

In [ ]:
test_data = pd.read_csv('/kaggle/input/project-1-layor-7/test.csv')
test_data.head()

In [ ]:
train_data.shape

### Checking for null values in the train data set.

In [ ]:
train_data.isnull().sum()

### Dropping the data in train data set for missing values.

In [ ]:
train_data = train_data.dropna()
train_data.isnull().sum()

In [ ]:
train_data.shape

## Train the data with all the features.

In [ ]:
X_train = train_data.drop(['label_1', 'label_2', 'label_3', 'label_4'], axis=1)
X_train.head()

In [ ]:
selected_label = 'label_1'
y_train = train_data[selected_label]
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_train

### Create and train the model using XGBoost.

In [ ]:
# Create an XGBoost classifier (or regressor for regression tasks)
model = xgb.XGBClassifier(verbosity=2, tree_method="gpu_hist")  # You can specify hyperparameters here

# Train the model on the training data
model.fit(X_train, y_train)

### Loading the valid data set.

In [ ]:
valid_data = pd.read_csv('/kaggle/input/project-1-layor-7/valid.csv')
valid_data.head()

#### Checking null values in the valid data set.

In [ ]:
valid_data.isnull().sum()

#### Dropping data for the missing values.

In [ ]:
valid_data = valid_data.dropna()
valid_data.isnull().sum()

In [ ]:
y_test = valid_data[selected_label]
valid_data = valid_data.iloc[:,:768]
valid_data.head()

### Make predictions on the valid data set.

In [ ]:
y_pred = model.predict(valid_data)
y_pred = le.inverse_transform(y_pred)
y_pred

### Evaluate the model.

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

## Create and train the dataset using SVC.

In [ ]:
# Creating the model.
svc_model = SVC(kernel='rbf', C=1.0)

#Train the model.
svc_model.fit(X_train, y_train)

### Make the predictions on the valid dataset.

In [ ]:
y_pred_svc = svc_model.predict(valid_data)
y_pred_svc = le.inverse_transform(y_pred_svc)
y_pred_svc

### Evaluate the SVC model.

In [ ]:
accuracy = accuracy_score(y_test, y_pred_svc)
print(f'Accuracy: {accuracy:.2f}')

**We got more accuracy from the SVC Model.**

## Testing the model.

### Checking if there any null values in the data set.

In [ ]:
test_data.isnull().sum()

In [ ]:
test_data.shape

In [ ]:
test_data.head()

In [ ]:
test_data = test_data.drop(['ID'],axis=1)
y_pred_test = svc_model.predict(test_data) # Using the svc_model since it has the more accuracy.
y_pred_test = le.inverse_transform(y_pred_test)
y_pred_test

In [ ]:
output_df = pd.DataFrame()
output_df[selected_label] = y_pred_test
output_df

In [ ]:
filename = selected_label + '.csv'
output_df.to_csv('/kaggle/working/'+filename, index=False)